# 04a - Vertex AI > Notebooks - Models Built in Notebooks with Tensorflow

Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

This notebook shows training a model directly within the runtime of the notebook environment.  Then the model is saved and moved to GCS for deployment to a Vertex AI Endpoint for online predictions.  The model training is done with [Tensorflow](https://www.tensorflow.org/), specifically [Keras](https://keras.io/), and was designed to show a neural network approach to logistic regression.  The training data batches are read from BigQuery using [Tensorflow I/O](https://www.tensorflow.org/io).

**Prerequisites:**

-  01 - BigQuery - Table Data Source

**Overview:**

-  Use Python Client for BigQuery
   -  Read the tables schema from BigQuery INFORMATION_SCHEMA
   -  Prepare the feature information for Tensorflow
-  Define a function that remaps the input data into features and target variables where target is one-hot encoded (classification model with 10 classes)
-  Set Tensorflow I/O read session
-  Demonstrate reading a single batch
-  Train a Tensorflow model
   -  Define the model layers
   -  Compile the model
   -  Fit the model
   -  Evaluate the model (loss, accuracy)
   -  Create prediction with the model
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Upload Model
      -  Model - aiplatform.Model.upoad
   -  Create Endpoint
      -  Endpoint - aiplatform.Endpoint.create
   -  Deploy to Endpoint
      -  Endpoint.deploy(model=Model)
   -  Online Predictions
      -  Endpoint.predict
-  Online Predictions with:
   -  REST call
   -  gcloud CLI

**Resources:**

-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)

**Related Training:**

-  todo

---
## Conceptual Architecture

<img src="architectures/statmike-mlops-04.png">

---
## Setup

inputs:

In [11]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'digits'
NOTEBOOK = '04a'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'

# Model Training
BATCH_SIZE = 30

packages:

In [12]:
from google.cloud import bigquery

from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import tensorflow as tf

from google.cloud import aiplatform
from datetime import datetime

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

clients:

In [13]:
aiplatform.init(project=PROJECT_ID, location=REGION)

parameters:

In [14]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
params = {"URI": URI}

---
## DELETE AFTER TESTING

In [48]:
PROJECT_ID='statmike-mlops'
REGION='us-central1'

BQDATASET_ID='digits'
BQTABLE_ID='digits_prepped'

MODEL_DIR='gs://{}/digits/model/04_keras'.format(PROJECT_ID)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

BATCH_SIZE = 30

MODEL_NAME='MODEL_04_KERAS-DIGITS'
ENDPOINT_NAME='ENDPOINT_04_KERAS-DIGITS'
params = {"MODEL_DIR":MODEL_DIR}
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'
DEPLOY_COMPUTE='n1-standard-4'

---
## Prepare Data Connection

Retrieve the Schema info from BigQuery Information Schema via the Storage API:
- https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas

In [15]:
bqclient = bigquery.Client()
bqjob = bqclient.query(
"""
SELECT * FROM `"""+DATANAME+""".INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
WHERE TABLE_NAME = '"""+DATANAME+"""' """
)
schema = bqjob.result().to_dataframe()
schema

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description
0,statmike-mlops,digits,digits,p0,p0,FLOAT64,None
1,statmike-mlops,digits,digits,p1,p1,FLOAT64,None
2,statmike-mlops,digits,digits,p2,p2,FLOAT64,None
3,statmike-mlops,digits,digits,p3,p3,FLOAT64,None
4,statmike-mlops,digits,digits,p4,p4,FLOAT64,None
...,...,...,...,...,...,...,...
61,statmike-mlops,digits,digits,p61,p61,FLOAT64,None
62,statmike-mlops,digits,digits,p62,p62,FLOAT64,None
63,statmike-mlops,digits,digits,p63,p63,FLOAT64,None
64,statmike-mlops,digits,digits,target,target,INT64,None


Use the the table schema to prepare the TensorFlow Model:
- Omit unused columns
- Create `feature_columns` for the model
- Define the `dtypes` for TensorFlow

In [16]:
OMIT = ['target_OE','splits']

selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != 'target':
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

from tensorflow.python.framework import dtypes
output_types = schema[~schema.column_name.isin(OMIT)].data_type.tolist()
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

Define a function that remaps the input data for TensorFlow into features, target and one_hot encodes the `target`:

In [17]:
def transTable(row_dict):
    target=row_dict.pop('target')
    target = tf.one_hot(tf.cast(target,tf.int64),10)
    target = tf.cast(target,tf.float32)
    return(row_dict,target)

Setup TensorFlow_IO client > session > table + table.map
- https://www.tensorflow.org/io/api_docs/python/tfio/bigquery/BigQueryClient

In [18]:
client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,DATANAME+'_prepped',DATANAME,selected_fields,output_types,row_restriction="splits='TRAIN'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)
train = table.shuffle(100000).batch(BATCH_SIZE)

In [19]:
client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,DATANAME+'_prepped',DATANAME,selected_fields,output_types,row_restriction="splits='TEST'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)
test = table.batch(BATCH_SIZE)

Review a single batch of the train data:

In [20]:
for a, b in train.take(1):
    columns=list(a.keys())
    print('columns: ',columns)
    print('target: ',b)

columns:  ['p0', 'p1', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p2', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p3', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p4', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p5', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p6', 'p60', 'p61', 'p62', 'p63', 'p7', 'p8', 'p9']
target:  tf.Tensor(
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0

---
## Train the Model In The Notebook

Define the Model:

In [56]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)
model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()],outputs=tf.keras.layers.Dense(10,activation=tf.nn.softmax)(feature_layer_outputs))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
#tf.keras.utils.plot_model(model,show_shapes=True, show_dtype=True)

In [57]:
#model.summary()

Fit the Model:

In [58]:
history = model.fit(train,epochs=25)

Epoch 1/25
49/49 [==============================] - 2s 4ms/step - loss: 3.1322 - accuracy: 0.5052
Epoch 2/25
49/49 [==============================] - 0s 4ms/step - loss: 0.7305 - accuracy: 0.8029
Epoch 3/25
49/49 [==============================] - 0s 4ms/step - loss: 0.4381 - accuracy: 0.8766
Epoch 4/25
49/49 [==============================] - 0s 3ms/step - loss: 0.3256 - accuracy: 0.8973
Epoch 5/25
49/49 [==============================] - 0s 3ms/step - loss: 0.2628 - accuracy: 0.9194
Epoch 6/25
49/49 [==============================] - 0s 4ms/step - loss: 0.2370 - accuracy: 0.9256
Epoch 7/25
49/49 [==============================] - 0s 3ms/step - loss: 0.1864 - accuracy: 0.9414
Epoch 8/25
49/49 [==============================] - 0s 3ms/step - loss: 0.1663 - accuracy: 0.9476
Epoch 9/25
49/49 [==============================] - 0s 4ms/step - loss: 0.1484 - accuracy: 0.9531
Epoch 10/25
49/49 [==============================] - 0s 4ms/step - loss: 0.1457 - accuracy: 0.9580
Epoch 11/25
49/49 [

Evaluate the model with the test data:

In [14]:
loss, accuracy = model.evaluate(test)

12/12 [==============================] - 1s 8ms/step - loss: 0.1644 - accuracy: 0.9480


Create Prediction from a batch of the test data:

In [15]:
model.predict(test.take(1))

array([[7.82250709e-09, 9.86453950e-01, 1.27303057e-08, 1.56892638e-04,
        2.61050189e-07, 1.28801773e-07, 1.47782542e-09, 1.90315474e-09,
        1.31083215e-02, 2.80541659e-04],
       [8.23077977e-12, 9.97530758e-01, 2.56055870e-07, 2.05361885e-05,
        5.31482328e-05, 1.34346506e-10, 1.72119219e-09, 7.44121820e-10,
        2.39505339e-03, 2.58356209e-07],
       [6.90522128e-08, 5.43578267e-01, 4.07441661e-07, 1.25681564e-01,
        3.08397591e-01, 4.94025549e-08, 8.13114675e-05, 3.32092139e-04,
        1.32961133e-02, 8.63250345e-03],
       [3.75077386e-10, 1.39880984e-04, 9.99859929e-01, 1.10227022e-10,
        2.89063995e-09, 4.79531027e-17, 2.72656203e-11, 2.96903668e-10,
        8.32370191e-08, 9.89527712e-16],
       [6.72645626e-07, 6.15355035e-04, 4.10887118e-13, 4.82091034e-10,
        9.99058545e-01, 1.79139392e-09, 3.13414406e-04, 2.74644799e-06,
        9.11851748e-06, 2.89242352e-10],
       [1.80979468e-11, 4.40917723e-03, 1.72594786e-10, 1.12222951e-08,
   

---
## Serving

### Upload The Model

In [17]:
model = aiplatform.Model.upload(
    display_name = MODEL_NAME,
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = MODEL_DIR
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/691911073727/locations/us-central1/models/7766237655074865152/operations/1377521768469626880
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/691911073727/locations/us-central1/models/7766237655074865152
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/691911073727/locations/us-central1/models/7766237655074865152')


In [18]:
model.display_name

'MODEL_04_KERAS-DIGITS'

### Create An Endpoint

In [19]:
endpoint = aiplatform.Endpoint.create(display_name = ENDPOINT_NAME)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/691911073727/locations/us-central1/endpoints/4241669569355186176/operations/6241409366029762560
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/691911073727/locations/us-central1/endpoints/4241669569355186176
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/691911073727/locations/us-central1/endpoints/4241669569355186176')


In [20]:
endpoint.display_name

'ENDPOINT_04_KERAS-DIGITS'

### Deploy Model To Endpoint

In [21]:
endpoint.deploy(
    model=model,
    deployed_model_display_name=MODEL_NAME+'_DEPLOYED',
    traffic_percentage = 100,
    machine_type = 'n1-standard-4',
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/691911073727/locations/us-central1/models/7766237655074865152 to Endpoint : projects/691911073727/locations/us-central1/endpoints/4241669569355186176
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/691911073727/locations/us-central1/endpoints/4241669569355186176/operations/3359105604512645120
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/691911073727/locations/us-central1/endpoints/4241669569355186176


---
## Prediction

### Data For Prediction

In [22]:
%%bigquery pred
SELECT *
FROM `digits.digits_source`
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  9.49rows/s]


In [23]:
pred

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p56,p57,p58,p59,p60,p61,p62,p63,target,target_OE
0,0.0,5.0,16.0,15.0,5.0,0.0,0.0,0.0,0.0,2.0,...,0.0,6.0,16.0,16.0,16.0,16.0,7.0,0.0,2,Even
1,0.0,5.0,16.0,12.0,1.0,0.0,0.0,0.0,0.0,5.0,...,0.0,8.0,16.0,16.0,16.0,16.0,4.0,0.0,2,Even
2,0.0,5.0,15.0,16.0,6.0,0.0,0.0,0.0,0.0,11.0,...,0.0,6.0,16.0,16.0,16.0,13.0,3.0,0.0,2,Even
3,0.0,4.0,15.0,15.0,8.0,0.0,0.0,0.0,0.0,8.0,...,0.0,7.0,14.0,11.0,0.0,0.0,0.0,0.0,2,Even
4,0.0,6.0,16.0,16.0,16.0,15.0,10.0,0.0,0.0,9.0,...,0.0,9.0,16.0,11.0,0.0,0.0,0.0,0.0,5,Odd
5,0.0,8.0,16.0,12.0,15.0,16.0,7.0,0.0,0.0,13.0,...,0.0,7.0,16.0,16.0,10.0,0.0,0.0,0.0,5,Odd
6,0.0,8.0,13.0,15.0,16.0,16.0,8.0,0.0,0.0,9.0,...,0.0,9.0,16.0,6.0,0.0,0.0,0.0,0.0,5,Odd
7,0.0,7.0,12.0,14.0,16.0,8.0,0.0,0.0,0.0,8.0,...,0.0,9.0,12.0,0.0,0.0,0.0,0.0,0.0,7,Odd
8,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0,Even
9,0.0,0.0,1.0,9.0,15.0,11.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,10.0,13.0,3.0,0.0,0.0,0,Even


### Prepare Prediction Request

In [24]:
pred.loc[:0]

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p56,p57,p58,p59,p60,p61,p62,p63,target,target_OE
0,0.0,5.0,16.0,15.0,5.0,0.0,0.0,0.0,0.0,2.0,...,0.0,6.0,16.0,16.0,16.0,16.0,7.0,0.0,2,Even


In [25]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')[0]

In [26]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [27]:
prediction = endpoint.predict(instances=instances,parameters=parameters)

In [28]:
prediction

Prediction(predictions=[[2.22375843e-06, 0.0271153804, 0.971013188, 8.32011392e-06, 1.18511302e-07, 2.80610698e-06, 9.4491852e-06, 2.84394162e-07, 0.00184659881, 1.69738792e-06]], deployed_model_id='4509097185509376000', explanations=None)

In [29]:
prediction.predictions[0]

[2.22375843e-06,
 0.0271153804,
 0.971013188,
 8.32011392e-06,
 1.18511302e-07,
 2.80610698e-06,
 9.4491852e-06,
 2.84394162e-07,
 0.00184659881,
 1.69738792e-06]

In [30]:
import numpy as np

np.argmax(prediction.predictions[0])

2

### Get Predictions: REST

In [36]:
import json
with open('request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [46]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      2.22375843e-06,
      0.0271153804,
      0.971013188,
      8.32011392e-06,
      1.18511302e-07,
      2.80610698e-06,
      9.4491852e-06,
      2.84394162e-07,
      0.00184659881,
      1.69738792e-06
    ]
  ],
  "deployedModelId": "4509097185509376000"
}


### Get Predictions: gcloud (CLI)

In [40]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request=request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[2.22375843e-06, 0.0271153804, 0.971013188, 8.32011392e-06, 1.18511302e-07, 2.80610698e-06, 9.4491852e-06, 2.84394162e-07, 0.00184659881, 1.69738792e-06]]


---
## Remove Resources
see notebook "XX - Cleanup"